unicode1

In [90]:
import itertools

In [60]:
import os

print(os.listdir("."))


['.envrc', 'flake.lock', 'cs336_basics.egg-info', 'LICENSE', 'cs336_basics', 'uv.lock', '.pytest_cache', 'CHANGELOG.md', 'pyproject.toml', 'tests', 'README.md', 'make_submission.sh', '.gitignore', 'flake.nix', '.venv', 'cs336_spring2025_assignment1_basics.pdf', '.git']


In [59]:
import builtins

In [52]:
import os
from typing import BinaryIO

In [56]:
import multiprocessing as mp

In [54]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")


/Users/adr/code/assignment1-basics/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21990/21990 [00:00<00:00, 739405.84 examples/s]


In [50]:
import regex as re

In [15]:
a = chr(0)
a

'\x00'

In [17]:
b = chr(0).__repr__()
b

"'\\x00'"

In [18]:
c = "this is a test" + chr(0) + "string"
c

'this is a test\x00string'

In [19]:
print(c)

this is a test string


`chr(0)` is ignored when printed.

training the tokenizer directly on unicode codepoints would be impractical, 
because the vocabulary would be very large and sparse.

Instead we will use a unicode encoding, the standard itself defines 3: UTF-8 UTF-16, and UTF-32. With UTF-8 being the dominant encoding for text on the internet.

In [28]:
test_string = "hello! こんにちは"
utf_8_encoded = test_string.encode("utf-8")
print(utf_8_encoded)

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf'


In [29]:
print(list(utf_8_encoded))

[104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175]


We go from a sequence of codepoints, to a sequence of bytes, where a byte is a number between 0 and 255.

In [35]:
utf_16_encoded = test_string.encode("utf-16")
print(utf_16_encoded)


b'\xff\xfeh\x00e\x00l\x00l\x00o\x00!\x00 \x00S0\x930k0a0o0'


In [36]:
print(list(utf_16_encoded))


[255, 254, 104, 0, 101, 0, 108, 0, 108, 0, 111, 0, 33, 0, 32, 0, 83, 48, 147, 48, 107, 48, 97, 48, 111, 48]


In [37]:
utf_32_encoded = test_string.encode("utf-32")
print(utf_32_encoded)


b'\xff\xfe\x00\x00h\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00!\x00\x00\x00 \x00\x00\x00S0\x00\x00\x930\x00\x00k0\x00\x00a0\x00\x00o0\x00\x00'


In [38]:
print(list(utf_32_encoded))


[255, 254, 0, 0, 104, 0, 0, 0, 101, 0, 0, 0, 108, 0, 0, 0, 108, 0, 0, 0, 111, 0, 0, 0, 33, 0, 0, 0, 32, 0, 0, 0, 83, 48, 0, 0, 147, 48, 0, 0, 107, 48, 0, 0, 97, 48, 0, 0, 111, 48, 0, 0]



we pick utf-8 for brevity? maybe. seem like this increases the size of the vocabulary?

In [39]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

In [41]:
decode_utf8_bytes_to_str_wrong(
    "hey".encode("utf-8")
)

'hey'

any unicode character that takes up more than one byte of storage.

In [46]:
bytes([104, 101]).decode("utf-8")

'he'

In [48]:
bad_bytes = [255,255]

byte-level tokenization slows down model training, A sentence with 10 words is 10 tokens long in a word-level language model, but it can be 50 or more tokens long in a character level model. A sub-token tokenizer trades off a larger vocabulary for shorter input streams. 
Byte-pair encoding iteratively merges the most frequently used pair of bytes with a single new unused index.

The process of constructing the BPE tokenizer vocabulary is known as *training* 

For efficiency, we pre-tokenize our corpus. 


In [49]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [51]:
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

In [53]:
def find_chunk_boundaries(file: BinaryIO, desired_num_chunks: int, split_special_token: bytes) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), "Must represent special token as a bytestring"

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks
    return sorted(set(chunk_boundaries))


In [91]:
with builtins.open("./cs336_basics/TinyStories-valid.txt", "rb") as f:
    boundaries = find_chunk_boundaries(f, 4, "<|endoftext|>".encode("utf-8"))
    print('boundaries', boundaries)
    # The following is a serial implementation, but you can parallelize this
    # by sending each start/end pair to a set of processes.
    for start, end in zip(boundaries[:-1], boundaries[1:]):
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")
        sub_chunks = re.split("|".join([re.escape("<|endoftext|>")]),chunk)
        # run tokenization for each of the sub_chunks.
        iterators = [re.finditer(PAT, sub_chunk) for sub_chunk in sub_chunks]
        all_matches = itertools.chain(*iterators)

boundaries [0, 4861876, 9724129, 14586631, 19447282]


In [92]:
all_matches

In [83]:
print(chunk[:100])
print(brev[:100])

<|endoftext|>

Lily and Ben are pupils in the same class. They like to play together, but they do no
['', '\n\nLily and Ben are pupils in the same class. They like to play together, but they do not like the same food. Lily likes spicy food, like curry and chili. Ben likes sweet food, like cake and candy.\nOne day, their teacher brings a big pot of soup for lunch. She says it is a special soup, with many different flavors. She tells the pupils to try some and share with their friends.\nLily takes a big spoon of soup and puts it in her mouth. She smiles and says, "Yum, this soup is spicy and good. I like it a lot." She gives some soup to Ben and says, "Here, try some. You will like it too."\nBen takes a small sip of soup and makes a face. He says, "Yuck, this soup is spicy and bad. I do not like it at all." He gives some soup to Lily and says, "Here, take it back. You can have it all."\nLily and Ben start to argue. They say, "My food is better than your food. Your food is gross and icky.

In [94]:
def pre_tokenize_chunk(chunk):
    sub_chunks = re.split("|".join([re.escape("<|endoftext|>")]),chunk)
    # run tokenization for each of the sub_chunks.
    iterators = [re.finditer(PAT, sub_chunk) for sub_chunk in sub_chunks]
    return itertools.chain(*iterators)

def train_bpe(input_path: str,vocab_size: int,special_tokens: list[str]):
    # open the input path
    with builtins.open(input_path, "rb") as f:
        # find the chunk boundaries
        boundaries = find_chunk_boundaries(f, 4, "<|endoftext|>".encode("utf-8"))
        chunks = []
        # TODO: parallelize
        for start, end in zip(boundaries[:-1], boundaries[1:]):
            # either parallelize by passing f 
            f.seek(start)
            chunk = f.read(end - start).decode("utf-8", errors="ignore")
            chunks.append(chunk)


        with mp.Pool() as pool: 
            results = pool.map(pre_tokenize_chunk, chunks)

        pre_tokenized = itertools.chain(*results) 

        return pre_tokenized
        


In [ ]:
all_matches = train_bpe("./cs336_basics/TinyStories-valid.txt", 4,[])
for i, match in enumerate(all_matches):
    if i >= 5:
        break
    print(match.group())

Process SpawnPoolWorker-6:
Traceback (most recent call last):
  File "/nix/store/v0vy1c59f0vf71h64caw17hzymvg0zms-python3-3.12.10/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/nix/store/v0vy1c59f0vf71h64caw17hzymvg0zms-python3-3.12.10/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/nix/store/v0vy1c59f0vf71h64caw17hzymvg0zms-python3-3.12.10/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/nix/store/v0vy1c59f0vf71h64caw17hzymvg0zms-python3-3.12.10/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'pre_tokenize_chunk' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Process SpawnPoolWorker-3:
Traceback (most recent call last):
  File "/nix/store/v0vy1c59f0vf71h64caw17hzymvg0zms-pytho

KeyboardInterrupt: 